<a href="https://colab.research.google.com/github/VDesi777/Practice/blob/main/Neural_Machine_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torchtext==0.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.0 MB 4.6 MB/s 
     |████████████████████████████████| 776.8 MB 4.9 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.0
    Uninstalling torchtext-0.14.0:
      Successfully uninstalled torchtext-0.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.0+cu116 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.


In [1]:
!python -m spacy download de_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.6 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [112]:
import torch

In [113]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset

In [114]:
import spacy

In [115]:
spacy_eng = spacy.load ('en_core_web_sm')

In [116]:
spacy_ger = spacy.load('de_core_news_sm')

In [117]:
def tokenize_eng(text):
  return [tok.text for tok in spacy_eng.tokenizer(text)]

In [118]:
def tokenize_ger(text):
  return [tok.text for tok in spacy_ger.tokenizer(text)]

In [119]:
english = Field(sequential=True, use_vocab=True, tokenize =tokenize_eng, lower=True,init_token='<sos>',eos_token='<eos>')
german = Field(sequential=True, use_vocab=True, tokenize =tokenize_ger, lower=True,init_token='<sos>',eos_token='<eos>')

In [120]:
train_data, validation_data, test_data = Multi30k.splits(exts=('.de','.en'),fields=(german,english))

In [121]:
english.build_vocab(train_data,max_size=10000,min_freq=2)
german.build_vocab(train_data,max_size=10000,min_freq=2)

In [122]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [123]:
train_iterator,validation_iterator,test_iterator = BucketIterator.splits(
    (train_data,validation_data,test_data),
    batch_size=64,
    device=device
)

In [124]:
for train in train_iterator:
  print(train)


[torchtext.data.batch.Batch of size 64 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 33x64 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 35x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 26x64 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 26x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 27x64 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 27x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 24x64 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 23x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 30x64 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 32x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 24x64 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 25x64 (GPU 0)]

[torchtext.data.batch

In [125]:
from torch import nn

In [126]:
import random

In [127]:
class Encoder(nn.Module):
  def __init__(self,input_size,embedding_size,hidden_size,num_layers,p):
    super(Encoder,self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.dropout = nn.Dropout(p)
    self.embedding = nn.Embedding(input_size,embedding_size)
    self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers,dropout = p)
  def forward(self,x): #x shape is (seq_len,N)
    embedding = self.dropout(self.embedding(x)) #embedding shape is (seq_len,N,embedding_size)
    outputs , (hidden,cell) = self.rnn(embedding) #hidden shape is (num_layers,N,hidden_size)
    return hidden , cell

In [128]:
class Decoder(nn.Module):
  def __init__(self,input_size,embedding_size,hidden_size,output_size,num_layers,p):
    super(Decoder,self).__init__()
    self.num_layers  = num_layers
    self.output_size = output_size
    self.hidden_size = hidden_size
    self.dropout = nn.Dropout(p)
    self.embedding = nn.Embedding(input_size,embedding_size)
    self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers,dropout=p)
    self.fc = nn.Linear(hidden_size,output_size)
  def forward(self,x,hidden,cell): #x shape is (N)
    x = x.unsqueeze(0) # now x shape is (1,N)
    embedding = self.dropout(self.embedding(x)) #embedding shape is (1,N,embedding_size)
    outputs , (hidden,cell) = self.rnn(embedding, (hidden,cell)) #outputs shape is (1,N,hidden_size)
    predictions = self.fc(outputs) #predictions shape is (1,N,english_vocab)
    predictions = predictions.squeeze(0) #now predictions shape is (N,english_vocab)
    return predictions, hidden ,cell

In [129]:
class Seq2Seq(nn.Module):
  def __init__(self,encoder,decoder,device):
    super(Seq2Seq, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self,src,trg,teacher_forcing_ratio=0.5):
    batch_size = trg.shape[1]
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_size
    
    outputs = torch.zeros(trg_len,batch_size,trg_vocab_size).to(self.device)
    

    hidden , cell = self.encoder(src)


    input = trg[0,:]
    
    for t in range(1,trg_len):
      output, hidden, cell = self.decoder(input,hidden,cell)
      outputs[t]=output
      teacher_force = random.random()<teacher_forcing_ratio
      top1 = output.argmax(1)
      input = trg[t] if teacher_force else top1
    
    return outputs #outputs shape is (trg_len,batch_size,english_vocab_size)
    

In [130]:
input_size = len(german.vocab)
output_size = len(english.vocab)
encoder_embedding_size = 256
decoder_embedding_size = 256
hidden_size = 512
num_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5

enc = Encoder(input_size,encoder_embedding_size,hidden_size,num_layers,encoder_dropout)
dec = Decoder(output_size,decoder_embedding_size,hidden_size,output_size,num_layers,decoder_dropout)

model = Seq2Seq(enc,dec,device).to(device)


In [131]:
def init_weights(m):
  for name, param in m.named_parameters():
    nn.init.uniform_(param.data, -0.08,0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=512, out_features=5893, bias=True)
  )
)

In [132]:
TRG_PAD_IDX = english.vocab.stoi[english.pad_token]
loss_func = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [133]:
optimizer = torch.optim.Adam(model.parameters())

In [134]:
def train(model,iterator,optimizer,loss_func,clip):
  model.train()
  epoch_loss=0

  for i,batch in enumerate(iterator):

    src = batch.src
    trg = batch.trg

    optimizer.zero_grad()
    output = model(src,trg,0.5)

    output_dim = output.shape[-1]

    output = output[1:].view(-1,output_dim)
    trg = trg[1:].view(-1)

    loss = loss_func(output,trg)
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(),clip)
    optimizer.step()

    epoch_loss+=loss.item()

  return epoch_loss/len(iterator)
  

In [138]:
def test(model,iterator,loss_func):
  model.eval()
  epoch_loss=0

  with torch.no_grad():

    for i,batch in enumerate(iterator):
      src = batch.src
      trg = batch.trg

      output = model(src,trg,0)

      output_dim = output.shape[-1]

      output = output[1:].view(-1,output_dim)
      trg = trg[1:].view(-1)

      loss = loss_func(output,trg)
      epoch_loss +=loss.item()
    return epoch_loss/len(iterator)

In [139]:
import time

In [140]:
EPOCHS=10
CLIP=1

best_valid_loss=float('inf')

for epoch in range(EPOCHS):
  start = time.time()

  train_loss = train(model,train_iterator,optimizer,loss_func,CLIP)
  valid_loss = test(model,validation_iterator,loss_func)

  end = time.time()

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(),'NMT.pt')

  print(f'Epoch: {epoch+1} | Time: {end-start} | Train Loss: {train_loss:.3f} | Validation Loss: {valid_loss:.3f}')

  

/usr/local/lib/python3.8/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 1 | Time: 44.369818449020386 | Train Loss: 4.151484077722491 | Validation Loss: 4.44886215031147
Epoch: 2 | Time: 44.14665508270264 | Train Loss: 3.821934388597631 | Validation Loss: 4.131429940462112
Epoch: 3 | Time: 43.72256398200989 | Train Loss: 3.569390748565943 | Validation Loss: 4.069793552160263
Epoch: 4 | Time: 45.00199365615845 | Train Loss: 3.384565508313116 | Validation Loss: 3.955677554011345
Epoch: 5 | Time: 44.49492907524109 | Train Loss: 3.209040032084293 | Validation Loss: 3.8416416198015213
Epoch: 6 | Time: 44.02756690979004 | Train Loss: 3.0665638977210428 | Validation Loss: 3.7371209263801575
Epoch: 7 | Time: 44.172014474868774 | Train Loss: 2.9104072070856977 | Validation Loss: 3.7095200270414352
Epoch: 8 | Time: 44.292688846588135 | Train Loss: 2.779477424033413 | Validation Loss: 3.727404475212097
Epoch: 9 | Time: 43.770580768585205 | Train Loss: 2.658536376932119 | Validation Loss: 3.569116309285164
Epoch: 10 | Time: 43.92575979232788 | Train Loss: 2.5364

In [142]:
model.load_state_dict(torch.load('NMT.pt'))
test_loss = test(model,test_iterator,loss_func)
print(f'Test Loss: {test_loss:.3f}')

/usr/local/lib/python3.8/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Test Loss: 3.589
